In [1]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output

In [2]:
# Function to generate the sine wave and play the sound
def generate_and_play_sound(amplitude, frequency, phase):
    # Time values for the sine wave
    duration = 2.0  # seconds
    sample_rate = 44100  # number of samples per second
    t = np.linspace(0.0, duration, int(duration * sample_rate), endpoint=False)

    # Generate the sine wave based on the provided parameters
    wave = amplitude * np.sin(2 * np.pi * frequency * t + phase)

    # Play the sound
    display(Audio(wave, rate=sample_rate))

    # Plot the sine wave
    plt.figure(figsize=(15, 8))
    plt.plot(t, wave)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title('Generated Sine Wave')
    plt.ylim(-5, 5)
    plt.xlim(0, 0.05)
    plt.grid(True)
    plt.show()


# Create the interactive widgets
amplitude_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Amplitude:')
frequency_slider = widgets.FloatSlider(value=440.0, min=20.0, max=2000.0, step=10.0, description='Frequency (Hz):')
phase_slider = widgets.FloatSlider(value=0.0, min=0.0, max=2*np.pi, step=0.1, description='Phase (radians):')

# Combine the widgets and the function
interactive_plot = widgets.interactive_output(generate_and_play_sound,
                                              {'amplitude': amplitude_slider,
                                               'frequency': frequency_slider,
                                               'phase': phase_slider})

# Display the interactive plot and widgets
widgets.VBox([amplitude_slider, frequency_slider, phase_slider, interactive_plot])


In [5]:
def generate_sine_wave(frequency, duration, amplitude=1.0, sample_rate=44100):
    t = np.linspace(0.0, duration, int(duration * sample_rate), endpoint=False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

# Create buttons and sliders to set the frequency and amplitude of sine components
num_components = 6  # Number of sine components
freq_sliders = [widgets.FloatSlider(value=440.0, min=0.0, max=10000.0, step=1.0, description=f'Frequency {i+1} (Hz):')
                for i in range(num_components)]
amp_sliders = [widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description=f'Amplitude {i+1}:')
               for i in range(num_components)]

def on_button_clicked(change):
    with output:
        clear_output(wait=True)
        frequencies = [slider.value for slider in freq_sliders]
        amplitudes = [slider.value for slider in amp_sliders]
        duration = 3.0  # seconds
        sample_rate = 44100  # number of samples per second
        t = np.linspace(0.0, duration, int(duration * sample_rate), endpoint=False)

        # Calculate the sum of sine waves with specified frequencies and amplitudes
        y_sum = np.sum([generate_sine_wave(freq, duration, amplitude=amp, sample_rate=sample_rate)
                        for freq, amp in zip(frequencies, amplitudes)], axis=0)

        display(Audio(y_sum, rate=sample_rate))
        plot_wave(t, y_sum)


def plot_wave(t, wave):
    sample_rate = 44100
    duration = len(wave) / sample_rate
    # t = np.linspace(0.0, duration, len(wave), endpoint=False)
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))
    axs[0].plot(t, wave)
    axs[0].set_xlabel('Time (s)')
    axs[0].set_ylabel('Amplitude')
    axs[0].set_title('Generated Sound Waveform')
    axs[0].set_xlim(0, 0.02)
    axs[0].grid(True)

    # Compute the FFT
    fft_values = 2 * np.fft.rfft(wave, norm="forward")
    freqs = np.fft.rfftfreq(len(wave), d=1.0/sample_rate)
    axs[1].plot(freqs, np.abs(fft_values))
    axs[1].set_xlabel('Frequency (Hz)')
    axs[1].set_ylabel('Magnitude')
    axs[1].set_title('Another Representation')
    axs[1].set_xlim(0, 10000)
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

# Assign event handlers to buttons
for slider in freq_sliders + amp_sliders:
    slider.observe(on_button_clicked, names='value')

# Display the widgets
controls_left = widgets.VBox(freq_sliders)
controls_right = widgets.VBox(amp_sliders)
output = widgets.Output()
display(widgets.HBox([controls_left, controls_right]), output)


Output()

In [6]:
def karplus_strong_string(frequency, duration, decay_factor=1-1e-2, sample_rate=44100):
    t = np.linspace(0.0, duration, int(duration * sample_rate), endpoint=False)
    n = int(sample_rate / frequency)
    wave = np.random.uniform(-1, 1, n)
    for i in range(len(t) - n):
        sample = decay_factor * 0.5 * (wave[i] + wave[i + 1])
        wave = np.append(wave, sample)
    return wave

# Guitar string frequencies (E2, A2, D3, G3, B3, E4)
guitar_frequencies = [82.41, 110.0, 146.83, 196.0, 246.94, 329.63]

# Create toggle buttons for guitar notes
note_buttons = [widgets.ToggleButton(description=f'Note {i}', value=False) for i in range(1, 7)]

def on_note_button_clicked(change):
    with output:
        clear_output(wait=True)
        # Update the sum of selected notes
        notes = [freq for freq, button in zip(guitar_frequencies, note_buttons) if button.value]
        if len(notes) > 0:
            guitar_sound = np.sum([karplus_strong_string(freq, duration=2.0) for freq in notes], axis=0)
            display(Audio(guitar_sound, rate=44100))
            plot_wave(guitar_sound)

def plot_wave(wave):
    sample_rate = 44100
    duration = len(wave) / sample_rate
    t = np.linspace(0.0, duration, len(wave), endpoint=False)
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))
    axs[0].plot(t, wave)
    axs[0].set_xlabel('Time (s)')
    axs[0].set_ylabel('Amplitude')
    axs[0].set_title('Generated Sound Waveform')
    axs[0].set_xlim(0, 0.05)
    axs[0].grid(True)

    # Compute the FFT
    fft_values = 2 * np.fft.rfft(wave, norm="forward")
    freqs = np.fft.rfftfreq(len(wave), d=1.0/sample_rate)
    axs[1].plot(freqs, np.abs(fft_values))
    axs[1].set_xlabel('Frequency (Hz)')
    axs[1].set_ylabel('Magnitude')
    axs[1].set_title('Another Representation')
    axs[1].set_xlim(0, 2000)
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

# Assign event handlers to buttons
for button in note_buttons:
    button.observe(on_note_button_clicked, names='value')

# Display the widgets
note_buttons_box = widgets.HBox(note_buttons)
output = widgets.Output()
display(note_buttons_box, output)


Output()